In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install deepdanbooru tensorflow-io huggingface_hub

In [2]:
from __future__ import annotations

import os
import pathlib
import tarfile

import deepdanbooru as dd
import huggingface_hub
import numpy as np
import PIL.Image
import tensorflow as tf

datasets_path = "/content/images" # @param {type:"string"}
Score_threshold = 0.5 # @param {type:"slider", min:0.1, max:1, step:0.01}

def load_sample_image_paths() -> list[pathlib.Path]:
    image_dir = pathlib.Path(datasets_path)
    return sorted(image_dir.glob('*'))


def load_model() -> tf.keras.Model:
    path = huggingface_hub.hf_hub_download('tensor-diffusion/DeepDanbooru-tagger',
                                           'model-resnet_custom_v3.h5')
    model = tf.keras.models.load_model(path)
    return model


def load_labels() -> list[str]:
    path = huggingface_hub.hf_hub_download('tensor-diffusion/DeepDanbooru-tagger',
                                           'tags.txt')
    with open(path) as f:
        labels = [line.strip() for line in f.readlines()]
    return labels


def predict(image: PIL.Image.Image, score_threshold: float, model: tf.keras.Model, labels: list[str]) -> tuple[dict[str, float], dict[str, float], str]:
    _, height, width, _ = model.input_shape
    image = np.asarray(image)
    image = tf.image.resize(image,
                            size=(height, width),
                            method=tf.image.ResizeMethod.AREA,
                            preserve_aspect_ratio=True)
    image = image.numpy()
    image = dd.image.transform_and_pad_image(image, width, height)
    image = image / 255.
    probs = model.predict(image[None, ...])[0]
    probs = probs.astype(float)

    indices = np.argsort(probs)[::-1]
    result_all = dict()
    result_threshold = dict()
    for index in indices:
        label = labels[index]
        prob = probs[index]
        result_all[label] = prob
        if prob < score_threshold:
            break
        result_threshold[label] = prob
    result_text = ', '.join(result_all.keys())
    return result_threshold, result_all, result_text

def process_and_save_batch(image_paths: list[pathlib.Path], score_threshold: float):
    for image_path in image_paths:
      try:
            # Check if the file has an allowed extension
            if image_path.suffix.lower() not in {'.jpg', '.jpeg', '.png'}:
                print(f"Skipping non-image file: {image_path}")
                continue
            # Load the image
            image = PIL.Image.open(image_path)

            if image.mode == 'RGBA':
              image = image.convert('RGB')

            # Run the prediction
            result_threshold, result_all, result_text = predict(image, score_threshold, model, labels)

            result_text = result_text.replace("rating:safe, ", "")

            # Print the results
            print("Image Path:", image_path)
            print("Result Threshold:", result_threshold)
            print("Result All:", result_all)
            print("Result Text:", result_text)
            print("\n")

            # Save the result_text to a text file
            result_file_path = image_path.with_suffix('.txt')
            with open(result_file_path, 'w') as result_file:
                result_file.write(str(result_text))

      except (PIL.UnidentifiedImageError, OSError) as e:
        # Handle UnidentifiedImageError or OSError
        print(f"Error processing image {image_path}: {e}")
        continue


image_paths = load_sample_image_paths()
model = load_model()
labels = load_labels()
score_threshold = Score_threshold

process_and_save_batch(image_paths, score_threshold)


model-resnet_custom_v3.h5:   0%|          | 0.00/624M [00:00<?, ?B/s]

(…)eepDanbooru-tagger/resolve/main/tags.txt:   0%|          | 0.00/105k [00:00<?, ?B/s]

1/1 [==============================] - 13s 13s/step
Image Path: /content/images/pincase2023  (165).png
Result Threshold: {'abigail_williams_(fate/grand_order)': 0.9999983310699463, 'polka_dot': 0.9999960660934448, 'polka_dot_bow': 0.9999387264251709, 'rating:safe': 0.999723494052887, 'stuffed_animal': 0.9996086955070496, 'stuffed_toy': 0.9995104074478149, 'bow': 0.9989004135131836, 'teddy_bear': 0.9964993000030518, 'orange_bow': 0.99647057056427, '1girl': 0.9945868253707886, 'blonde_hair': 0.9915307760238647, 'hat': 0.9879969954490662, 'hair_bow': 0.983740508556366, 'bangs': 0.9766174554824829, 'parted_bangs': 0.9697731733322144, 'black_headwear': 0.9691610336303711, 'solo': 0.9689008593559265, 'black_bow': 0.9674030542373657, 'sleeves_past_wrists': 0.9636602997779846, 'blue_eyes': 0.9565430283546448, 'covered_mouth': 0.9557707905769348, 'long_sleeves': 0.946300208568573, 'long_hair': 0.9359219074249268, 'dress': 0.8893999457359314, 'looking_at_viewer': 0.8313681483268738, 'eyebrows_vi

In [4]:
#@title Download datasets with txt file
import os
from google.colab import files
import shutil

folder_path = datasets_path

file_type = "txt only" # @param ["txt only", "img + txt"]

if file_type == "txt only":
  format = '.txt'
elif file_type == "img + txt":
  format = '.png', '.jpg', '.jpeg', '.gif', ".txt","json"

def zip_and_download_images(folder_path, zip_filename='datasets.zip'):
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} not found.")
        return

    file_list = os.listdir(folder_path)

    temp_folder = '/content/datasets'
    os.makedirs(temp_folder, exist_ok=True)

    for file_name in file_list:
        if file_name.lower().endswith((format)):
            file_path = os.path.join(folder_path, file_name)
            shutil.copy(file_path, temp_folder)

    shutil.make_archive(temp_folder, 'zip', temp_folder)

    print(f"Downloading ZIP: {zip_filename}")
    files.download(f'{temp_folder}.zip')

    shutil.rmtree(temp_folder)

zip_and_download_images(folder_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  

**Diffusion Pipeline**

Huggingface: https://huggingface.co/DamarJati

Github: https://github.com/Damarcreative

Instagram: @Damar_shop & @Pixonelab

Mail: dev@damarcreative.my.id

Site: www.damarcreative.my.id

------
